In [7]:
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import sqlalchemy as sq
import seaborn as sns
import pandas as pd
import numpy as np
import os, sys

sys.path.append("../")
from Shared.DataService import DataService

In [8]:
load_dotenv()
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [9]:
if (
    PG_DB is None
    or PG_ADDR is None
    or PG_PORT is None
    or PG_USER is None
    or PG_PW is None
):
    raise ValueError("Environment variables not set")

db = DataService(PG_DB, PG_ADDR, int(PG_PORT), PG_USER, PG_PW)
conn = db.connect()

In [15]:
dlyDF = pd.read_csv('data/aggregatedDly.csv')

In [18]:
ergotQuery = sq.text(
    """
    SELECT year, district, percnt_true, has_ergot, sum_severity, present_prev1, present_prev2, present_prev3, present_in_neighbor, severity_prev1, 
        severity_prev2, severity_prev3, severity_in_neighbor FROM public.agg_ergot_samples
    GROUP BY year, district, percnt_true, has_ergot, sum_severity, present_prev1, present_prev2, present_prev3, present_in_neighbor, severity_prev1, 
        severity_prev2, severity_prev3, severity_in_neighbor;
    """
)

ergotDF = pd.read_sql_query(ergotQuery, conn)

In [35]:
final_df = dlyDF.merge(ergotDF, on=["year", "district"])
final_df.drop(columns=['Unnamed: 0'], inplace=True)

In [36]:
final_df

,year,district,01-01:max_temp,01-01:min_temp,01-01:mean_temp,01-01:min_total_rain,01-01:max_total_rain,01-01:mean_total_rain,01-01:min_total_snow,01-01:max_total_snow,...,has_ergot,sum_severity,present_prev1,present_prev2,present_prev3,present_in_neighbor,severity_prev1,severity_prev2,severity_prev3,severity_in_neighbor
0,1995,4860,-8.0375,-20.200000,-14.137500,0.0,0.0,0.0,0.0,2.0,...,False,0.00,False,False,False,True,0.000000,0.000000,0.000000,0.003682
1,1995,4810,-7.4200,-17.000000,-12.220000,0.0,0.0,0.0,0.0,0.5,...,False,0.00,False,False,False,False,0.000000,0.000000,0.000000,0.000000
2,1995,4840,-8.3000,-19.425000,-13.875000,0.0,0.0,0.0,0.0,4.2,...,True,15.00,False,False,False,False,0.000000,0.000000,0.000000,0.000000
3,1995,4830,-5.7500,-20.258333,-13.033333,0.0,0.0,0.0,0.0,0.0,...,True,11.00,False,False,False,True,0.000000,0.000000,0.000000,0.001817
4,1995,4731,-7.3000,-19.650000,-13.500000,0.0,0.0,0.0,0.0,0.0,...,True,6.00,False,False,False,True,0.000000,0.000000,0.000000,0.013627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,2022,4605,-28.3000,-33.000000,-30.700000,0.0,0.0,0.0,0.0,0.0,...,True,0.05,True,True,True,True,0.012821,0.089744,0.009901,0.298913
1032,2022,4720,-24.8000,-36.900000,-30.850000,0.0,0.0,0.0,0.0,0.0,...,True,0.32,True,True,True,True,0.042553,0.322581,0.080808,0.623134
1033,2022,4611,-25.8000,-35.600000,-30.700000,0.0,0.0,0.0,0.0,0.0,...,True,0.07,False,True,True,True,0.000000,0.235294,0.043478,0.325397
1034,2022,4751,-22.9000,-33.600000,-28.300000,0.0,0.0,0.0,0.0,0.0,...,True,0.53,True,True,True,True,0.037736,0.327044,0.033520,0.387460


In [21]:
# Basic correlogram
sns.pairplot(final_df)
plt.show()

KeyboardInterrupt: 

Error in callback <function _draw_all_if_interactive at 0x0000025C5B648AF0> (for post_execute):


ValueError: Image size of 1369500x1369500 pixels is too large. It must be less than 2^16 in each direction.

ValueError: Image size of 1369500x1369500 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 1.3695e+06x1.3695e+06 with 797 Axes>

In [ ]:
sns.set_theme(style="white")

# Compute the correlation matrix
corr = df.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})